In [1]:
# Logistic Regression 구현
# Wisconsin Breast Cancer Data

# 필요한 module import 
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

Matplotlib created a temporary config/cache directory at C:\Users\lee_0\AppData\Local\Temp\matplotlib-z4s9e1wb because the default path (C:\Users\lee_0\.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
# Raw Data Loading
cancer = load_breast_cancer()
# x_data(feature), t_data(0, 1) 데이터의 설명...

# x_data => cancer.data
# t_data => cancer.target
# print(cancer.data.shape, cancer.target.shape)
# (569, 30) => x_data는 총 569개의 행으로 구성, 컬럼(feature)은 30개
# (569,) => t_data 역시 총 569개 있어요. 0과 1로 구성되어 있어요!

# [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 1]
np.unique(cancer.target, return_counts=True)
# (array([0, 1]), array([212, 357], dtype=int64))
#                                0.37, 0.63
# 0은 악성종양(나쁜거), 1은 양성종양(괜찮은거)
# 약간의 데이터 불균형이 존재(imbalanced data)

# 데이터셋 저장
x_data = cancer.data
t_data = cancer.target

# boxplot을 이용해서 이상치와 데이터 분포를 간단하게 확인
# plt.boxplot(x_data)
# plt.show()

# 데이터 정규화가 필요해요!
# 원래 정규화는 당연히 이상치를 제거하고 진행하는게 맞아요!
# 실제적인 이상치는 존재하지 않는다고 가정하고 진행!
scaler = MinMaxScaler()
scaler.fit(x_data)
x_data_norm = scaler.transform(x_data)

# plt.boxplot(x_data_norm)
# plt.show()

# 데이터를 분리해야 해요! 학습용과 평가용으로 분리
# train data와 test data로 분리(데이터를 섞어서 분리)
x_data_train_norm, x_data_test_norm, t_data_train, t_data_test = \
train_test_split(x_data_norm,
                 t_data,
                 stratify=t_data,
                 test_size=0.2,  # default값은 0.25
                 random_state=3)

# 섞어서 분리했기 때문에 데이터의 편향이 없을거라 생각되는데
# 확인은 해야 겠죠.
np.unique(t_data_test, return_counts=True)
# (array([0, 1]), array([42, 72], dtype=int64)) 0.35, 0.65

(array([0, 1]), array([42, 72], dtype=int64))

In [3]:
# sklearn model구현

sklearn_model = linear_model.LogisticRegression()

# 학습하기 전에 cross validation을 한번 수행해 볼꺼예요!
# train data를 가지고 수행해요!
score = cross_val_score(sklearn_model,
                        x_data_train_norm,
                        t_data_train,
                        cv=5)

print(score)  # [0.97802198 0.94505495 0.95604396 0.98901099 0.94505495]
print(f'sklearn의 평균 validation accuracy : {np.mean(score)}')  # 0.9626373626373625

# model 학습진행
sklearn_model.fit(x_data_train_norm,
                  t_data_train)

# model 최종평가
test_score = sklearn_model.score(x_data_test_norm,
                                 t_data_test)
print(f'모델의 최종 score : {test_score}')  # 0.9649122807017544

[0.97802198 0.94505495 0.95604396 0.98901099 0.94505495]
sklearn의 평균 validation accuracy : 0.9626373626373625
모델의 최종 score : 0.9649122807017544


In [4]:
# Tensorflow 구현

keras_model = Sequential()

keras_model.add(Flatten(input_shape=(30,)))
keras_model.add(Dense(units=1,
                      activation='sigmoid'))

# 학습할 때 매 epoch마다 validation을 수행하고
# validation의 평가 기준은 accuracy를 사용하겠어요!
keras_model.compile(optimizer=Adam(learning_rate=1e-1),
                    loss='binary_crossentropy',
                    metrics=['acc'])

keras_model.fit(x_data_train_norm,
                t_data_train,
                epochs=300,
                verbose=1,
                validation_split=0.2)

# training data로 학습한 후
# training data로 평가 vs validation data로 평가

# loss : training data로 학습한 후 training data를 이용해서 계산한 loss
# val_loss : validation data로 계산한 loss

# learning_rate=1e-1
# loss: 0.0423 - acc: 0.9890 - val_loss: 0.0983 - val_acc: 0.9670

# learning_rate=1e-2
# loss: 0.0793 - acc: 0.9808 - val_loss: 0.0947 - val_acc: 0.9560

# learning_rate=1e-4
# loss: 0.5635 - acc: 0.8489 - val_loss: 0.5667 - val_acc: 0.8352

Epoch 1/300
12/12 [==============================] - 0s 14ms/step - loss: 0.4817 - acc: 0.8462 - val_loss: 0.3663 - val_acc: 0.8681
Epoch 2/300
12/12 [==============================] - 0s 3ms/step - loss: 0.3004 - acc: 0.9093 - val_loss: 0.2792 - val_acc: 0.8791
Epoch 3/300
12/12 [==============================] - 0s 2ms/step - loss: 0.2492 - acc: 0.9121 - val_loss: 0.2495 - val_acc: 0.9121
Epoch 4/300
12/12 [==============================] - 0s 2ms/step - loss: 0.2197 - acc: 0.9011 - val_loss: 0.2574 - val_acc: 0.8791
Epoch 5/300
12/12 [==============================] - 0s 2ms/step - loss: 0.1996 - acc: 0.9313 - val_loss: 0.2053 - val_acc: 0.9231
Epoch 6/300
12/12 [==============================] - 0s 2ms/step - loss: 0.1844 - acc: 0.9368 - val_loss: 0.1929 - val_acc: 0.9121
Epoch 7/300
12/12 [==============================] - 0s 2ms/step - loss: 0.1721 - acc: 0.9451 - val_loss: 0.1776 - val_acc: 0.9231
Epoch 8/300
12/12 [==============================] - 0s 2ms/step - loss: 0.1584 - 

In [5]:
# 학습이 다 끝났어요!
# Evaluation(평가)
result = keras_model.evaluate(x_data_test_norm,
                              t_data_test)

print(result)
# loss 값                      , accuracy 값
# [0.13803397119045258, 0.9649122953414917]

4/4 [==============================] - 0s 2ms/step - loss: 0.1378 - acc: 0.9649
[0.1377609223127365, 0.9649122953414917]


In [6]:
# Titanic data를 이용한 Logistic Regression 구현
# Kaggle에 있는 데이터를 이용
# Kaggle에 있는 데이터를 전처리 한 후
# 모델을 만들어서 학습을 진행
# 자체 평가를 진행(validation)
# 모델을 이용해서 예측값을 추출(test.csv)
# 예측된 결과를 Kaggle에 upload해서 우리 모델의 성능을 검증

In [7]:
# Raw Data Loading
df = pd.read_csv(r"C:\Users\lee_0\Desktop\코딩\ML\12.05\train.csv")

# display(df)  # 891 rows × 12 columns

# 데이터 전처리!(feature engineering)
train = df

# 사용하는 column만 추출, 사용되지 않는(불필요한) column을 삭제
# print(train.columns) 
# Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
# 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype='object')
# 필요없는 컬럼은(종속변수에 영향을 주지 않거나 의미상 중복 컬럼)

train.drop(['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin'],
           axis=1,
           inplace=True)
# display(df)  # 891 rows × 7 columns

# 성별처리 (male=0, female=1)
gender_mapping = {'male' : 0,
                  'female' : 1}
train['Sex'] = train['Sex'].map(gender_mapping)
# display(df)

train['Falmily'] = train['SibSp'] + train['Parch']
# display(df)  # 891 rows × 8 columns

train.drop(['SibSp', 'Parch'],
           axis=1,
           inplace=True)
# display(df)  # 891 rows × 6 columns

# df.info()

# Embarked의 결측치를 처리해 보아요!
train['Embarked'] = train['Embarked'].fillna('Q')

# Embarked 처리
Embarked_mapping = {'S' : 0,
                    'C' : 1,
                    'Q' : 2}
train['Embarked'] = train['Embarked'].map(Embarked_mapping)
# display(df)

# 나이를 처리해야 해요! 나이에는 결측치가 많아요!
train['Age'] = train['Age'].fillna(train['Age'].mean())
# display(df)

# 나이에 대해서는...Binning 처리를 해요!
train.loc[train['Age'] < 8, 'Age'] = 0
train.loc[(train['Age'] >= 8) & (train['Age'] < 20), 'Age'] = 1
train.loc[(train['Age'] >= 20) & (train['Age'] < 65), 'Age'] = 2
train.loc[train['Age'] >= 65, 'Age'] = 3
# display(df)

x_data = train.drop('Survived', axis=1, inplace=False).values
t_data = train['Survived'].values.reshape(-1, 1)

# 정규화!
scaler = MinMaxScaler()
scaler.fit(x_data)
x_data_norm = scaler.transform(x_data)

# train 데이터와 test 데이터를 분리하나요?
# 원래 내가 만든 모델의 최종 평가를 하기 위해서는 test 데이터가 당연히 있어야 해요!
# 하지만 우리 예제는 Kaggle에서 제공한 test 데이터를 이용한 예측결과값을
# 파일로 만들어서 Kaggle에 제출하는 것이기 때문에 test 데이터가 필요 없어요!

In [8]:
# Keras Model을 만들어 보아요!

keras_model = Sequential()

keras_model.add(Flatten(input_shape=(5,)))
keras_model.add(Dense(units=1,
                      activation='sigmoid'))

keras_model.compile(optimizer=Adam(learning_rate=1e-1),
                    loss='binary_crossentropy',
                    metrics=['acc'])

keras_model.fit(x_data_norm,
                t_data,
                epochs=300,
                verbose=1,
                validation_split=0.2)

# learning_rate=1e-1
# loss: 0.4606 - acc: 0.7978 - val_loss: 0.4007 - val_acc: 0.8268

# learning_rate=1e-2
# loss: 0.4538 - acc: 0.8006 - val_loss: 0.3906 - val_acc: 0.8212

# learning_rate=1e-4
# loss: 0.6656 - acc: 0.6110 - val_loss: 0.6291 - val_acc: 0.6536

Epoch 1/300
23/23 [==============================] - 0s 5ms/step - loss: 0.5620 - acc: 0.7247 - val_loss: 0.4412 - val_acc: 0.8101
Epoch 2/300
23/23 [==============================] - 0s 2ms/step - loss: 0.4707 - acc: 0.7949 - val_loss: 0.4227 - val_acc: 0.8101
Epoch 3/300
23/23 [==============================] - 0s 1ms/step - loss: 0.4672 - acc: 0.7949 - val_loss: 0.4194 - val_acc: 0.8101
Epoch 4/300
23/23 [==============================] - 0s 2ms/step - loss: 0.4656 - acc: 0.7907 - val_loss: 0.4137 - val_acc: 0.8212
Epoch 5/300
23/23 [==============================] - 0s 2ms/step - loss: 0.4662 - acc: 0.7949 - val_loss: 0.4125 - val_acc: 0.8156
Epoch 6/300
23/23 [==============================] - 0s 2ms/step - loss: 0.4630 - acc: 0.7921 - val_loss: 0.4093 - val_acc: 0.8156
Epoch 7/300
23/23 [==============================] - 0s 2ms/step - loss: 0.4610 - acc: 0.8062 - val_loss: 0.4252 - val_acc: 0.8045
Epoch 8/300
23/23 [==============================] - 0s 2ms/step - loss: 0.4722 - a

In [62]:
# 학습이 끝났으니... 제출파일을 생성해야 해요!
# 모델이 만들어 졌으니 test 데이터로 예측된 결과를 csv파일로
# 만들어서 Kaggle에 제출하고 그 결과를 확인해보아요!
test = pd.read_csv(r"C:\Users\lee_0\Desktop\코딩\ML\12.05\test.csv")
submission = pd.read_csv(r"C:\Users\lee_0\Desktop\코딩\ML\12.05\gender_submission.csv")

# test 데이터 전처리
# 사용하는 column만 추출, 사용되지 않는(불필요한) column을 삭제
print(train.columns)
# 필요없는 컬럼은(종속변수에 영향을 주지 않거나 의미상 중복 컬럼)

test.drop(['PassengerId', 'Ticket', 'Name', 'Fare', 'Cabin'],
          axis=1,
          inplace=True)

# 성별처리 (male=0, female=1)
gender_mapping = {'male' : 0,
                  'female' : 1}
test['Sex'] = test['Sex'].map(gender_mapping)

test['Family'] = test['SibSp'] + test['Parch']

test.drop(['SibSp', 'Parch'],
           axis=1,
           inplace=True)

# Embarked의 결측치를 처리해 보아요!
test['Embarked'] = test['Embarked'].fillna('Q')

# Embarked 처리
Embarked_mapping = {'S' : 0,
                    'C' : 1,
                    'Q' : 2}
test['Embarked'] = test['Embarked'].map(Embarked_mapping)

# 나이를 처리해야 해요! 나이에는 결측치가 많아요!
test['Age'] = test['Age'].fillna(test['Age'].mean())

# 나이에 대해서는...Binning 처리를 해요!
test.loc[test['Age'] < 8, 'Age'] = 0
test.loc[(test['Age'] >= 8) & (test['Age'] < 20), 'Age'] = 1
test.loc[(test['Age'] >= 20) & (test['Age'] < 65), 'Age'] = 2
test.loc[test['Age'] >= 65, 'Age'] = 3
# display(test)  # 418 rows × 5 columns

x_data_test_norm = scaler.transform(test.values)

predict = keras_model.predict(x_data_test_norm)

submission['Survived'] = predict
submission['Survived'] = np.where(submission['Survived'] >= 0.5, 1, 0)
# display(submission)  # 418 rows × 2 columns

submission.to_csv('./sub.csv', index=False)

Index(['Survived', 'Pclass', 'Sex', 'Age', 'Embarked', 'Falmily'], dtype='object')
14/14 [==============================] - 0s 734us/step
